In [71]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px

import logging
import os
import nltk
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Indeed 

In [2]:
df = pd.read_csv("indeedcoJD.csv")

In [6]:
df.to_csv("indeedcoJD.csv")

In [4]:
df.drop(columns='Unnamed: 0',inplace = True)

In [5]:
df

,Jobtitle,Company,City,State,FormattedLocation,Date,Snippet,URL,Latitude,Longitude,Salary,Rating,full_jd
0,Night Fill Team Member,Woolworths Supermarkets,Para Hills,SA,Para Hills SA,"Mon, 28 Sep 2020 11:14:33 GMT","Whether you’re starting out, already on the wa...",https://au.indeed.com/viewjob?jk=7d109f7660adf...,-34.79980,138.66250,None,3.8,Night Fill Team Member\n\nStarting in the afte...
1,Retail Assistant (20 hrs pw),ALDI,Dandenong,VIC,Dandenong VIC,"Mon, 28 Sep 2020 03:50:31 GMT",A Retail Assistant at ALDI needs to be able to...,https://au.indeed.com/viewjob?jk=9d1505be67b54...,-37.98333,145.20000,None,3.8,"At ALDI, our people are the key to our success..."
2,Fresh Team Member,Woolworths Supermarkets,Springfield,QLD,Springfield QLD,"Mon, 28 Sep 2020 11:14:32 GMT","Whether you’re starting out, already on the wa...",https://au.indeed.com/viewjob?jk=5f6559ba4dff2...,-27.66390,152.89700,None,3.8,Fresh Team Member\n\nFrom meats and cheeses to...
3,"Fruit Picking, Thinning, Pruning, Packing",Everlasting Agriculture Management,NaN,WA,Western Australia,"Fri, 30 Aug 2019 04:33:55 GMT",Work will be paid accordingly to casual minimu...,https://au.indeed.com/viewjob?jk=a6fe75fb5c3d5...,NaN,NaN,None,3.8,We are always looking for people to join our t...
4,Drive-Thru Attendant,BWS,Gawler,SA,Gawler SA,"Tue, 29 Sep 2020 05:14:29 GMT",Whether it's in the way we connect with custom...,https://au.indeed.com/viewjob?jk=896b1d74f6720...,-34.60000,138.73330,None,3.8,"At BWS, we are passionate about Beer, Wine and..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Courts Management Officer,Queensland Government,Brisbane,QLD,Brisbane QLD,"Mon, 28 Sep 2020 10:41:35 GMT",Applications to remain current for 12 months. ...,https://au.indeed.com/viewjob?jk=05e2b8d1e26bc...,-27.46794,153.02809,NaN,NaN,NaN
5196,Perth DFO - Christmas Casuals,City Beach Australia,Perth,WA,Perth WA,"Tue, 29 Sep 2020 03:07:40 GMT",A Behind the Scenes shift is performed outside...,https://au.indeed.com/viewjob?jk=f82ca33525858...,-31.93333,115.83333,NaN,NaN,NaN
5197,Peter Alexander,The Just Group,Sydney Western Suburbs,NSW,Sydney Western Suburbs NSW,"Fri, 07 Aug 2020 08:45:13 GMT",All the training to get you started in Fashion...,https://au.indeed.com/viewjob?jk=9187827d91f75...,-33.79840,150.92660,NaN,NaN,NaN
5198,Christmas Casual Sales Assistant Supre Green H...,Cotton On,Green Hills,NSW,Green Hills NSW,"Mon, 14 Sep 2020 08:26:48 GMT",You're the key to a winning store. With 7 bran...,https://au.indeed.com/viewjob?jk=a705b8bfdb8a1...,-32.87080,151.51880,NaN,NaN,NaN


In [20]:
df.isnull().sum()

Jobtitle                0
Company                25
City                  858
State                 498
FormattedLocation       0
Date                    0
Snippet                 0
URL                     0
Latitude              858
Longitude             858
Salary               3805
Rating               3805
full_jd              4451
dtype: int64

In [29]:
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", zoom = 3, 
                        color = 'Rating', color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max = 10, mapbox_style="open-street-map")

fig.show()

** Concentrata at 4 main cities: Adelaide, Melbourne, Sydney, Brisbane.
Among that, Melbourne seems to have the highest concentration of job inquiry

In term of working quality, the map illustrates a marginal difference between states where hot color represent higher ratings. Melbourne can be inferred as the most desirable working city across the nation followed by Sydney and Brisbane.

In regional areas, there appears to be a considerable job demand level as opposed to our initial expectation. In the near future, we hope to foster a contribution of urbanizing these areas. **

# Resume samples 

In [85]:
df_r = pd.read_csv("resume_samples_10000.csv")

In [90]:
df_r.drop(columns='Unnamed: 8',inplace=True)

In [79]:
df_r.dropna(inplace=True)

In [80]:
def tags_from_text( reviews,
                    blacklist,
                    bigram_weight=2, 
                    trigram_weight=3,
                    ngram_range=[2,4],
                    stop_words=None):
    
    tv=CountVectorizer(ngram_range=ngram_range, stop_words=stop_words, encoding='utf8')
    tv.fit(reviews)
    
    review_vec = tv.transform(reviews)
    n_reviews = len(reviews)*1.0
    #print review_vec
    feats = tv.get_feature_names()
    #print feats
    counts = np.squeeze(np.asarray(review_vec.sum(axis=0)))
    feat_scores = {}
    dict = {'feature':[], 'score':[],'count':[]}
    #print counts
    for feat, count  in zip(feats, counts):
        ngram_score = 1
        #print "****", len(feat.split())
        if not(feat.endswith('skill') 
               or feat.endswith('skills')
               or feat.startswith('perform') 
               or feat.startswith('deliver')
               or feat.startswith('ability')
               or feat.startswith('avail')
               or feat.endswith('experience')
               or feat.startswith('demonstrate')
               or 'knowledge' in feat
               or 'licen' in feat 
               or 'educat' in feat
               or 'able' in feat
               or 'cert' in feat):
            continue
            
        # if len(np.intersect1d(['skill','skills'], feat.split()[-1])) == 0:
        #    continue
        if len(feat.split())>1:
            ngram_score = bigram_weight
        if len(feat.split())>2:
            ngram_score = trigram_weight
        if len(feat.split())>3:
            ngram_score = 27
            #print "***",  ngram_score, feat 
        score = count**ngram_score
    
        if len(np.intersect1d(blacklist, feat.split())) > 0:
            score = 0.
        if score> 0:
            feat_scores[feat] =  score
            dict['feature'] += [feat]
            dict['score'] += [score]
            dict['count'] += [count]
    return pd.DataFrame(dict)

In [95]:
blacklist = []
score_features = tags_from_text(df_r['SKILLS'].values, blacklist)

In [98]:
score_features.sort_values('score', ascending=False, inplace=True)
score_features['norm_score'] = score_features['score']/score_features.score.max()

In [100]:
score_features[score_features['count']>50].sort_values('count',ascending=False)

,feature,score,count,norm_score
4687,computer skills,1075369,1037,1.170255e-13
667,accounts payable,203401,451,2.213483e-14
818,accounts receivable,80656,284,8.777278e-15
2331,basic computer skills,18609625,265,2.025167e-12
14273,retail skills,67600,260,7.356476e-15
8740,knowledge of,55225,235,6.009784e-15
15885,table setup,42025,205,4.573312e-15
11865,payable accounts,41209,203,4.484512e-15
673,accounts payable accounts,7645373,197,8.319971e-13
11871,payable accounts receivable,7189057,193,7.823392e-13


In [154]:
for url in df['URL'][749:]:
    connection = requests.get(url)
    soup = BeautifulSoup(connection.text, 'html.parser')
    jd = soup.find(id = 'jobDescriptionText')
    if jd == None:
        break
    jds.append(jd.text)
